In [1]:
from pathlib import Path
import neat, visualize
from neat.parallel import ParallelEvaluator
from pprint import pprint

from game_environment import BatchEvaluator, cache_directory_files

In [2]:

def add_file():
    if len(working_files) == 0:
        raise ValueError("No more files in data cache")
    
    file = working_files.pop()
    data = data_cache.pop(file)
    working_cache[file] = data
    
data_dir = r"../Cirricula/Phase 1"
data_cache = cache_directory_files(data_dir,None, 1 )

working_files = list(data_cache.keys())
num_working_files = len(working_files)
working_cache = {}

# === Training hyperparameters ===
frequency            = 10    # generations per curriculum step
data_growth_factor   = 2      # double files each step
num_frequency_steps  = 10      # how many curriculum expansions
num_look_back_period = 10     # environment parameter

In [ ]:
# === NEAT run ===
def run(config_path: Path):
    # Load NEAT configuration
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        str(config_path)
    )

    # Create population and reporters
    pop = neat.Population(config)
    pop.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    pop.add_reporter(stats)
    pop.add_reporter(neat.Checkpointer(frequency))

    # Instantiate vectorized evaluator (no ParallelEvaluator)
    evaluator = BatchEvaluator(working_cache, num_look_back_period)

    winner = None
    files_to_add = 1

    # Curriculum loop
    for step in range(num_frequency_steps):
        # Expand curriculum: double files
        files_to_add = min(len(working_files), files_to_add)
        for _ in range(files_to_add):
            add_file()
        files_to_add *= data_growth_factor

        # Update evaluator's cache
        evaluator._data_cache = working_cache
        evaluator._files = list(working_cache.keys())
        print(f"Curriculum step {step}: now using {len(working_cache)} files.")

        # Run NEAT for 'frequency' generations using vectorized evaluator
        winner = pop.run(evaluator, frequency)

    # Final best genome
    print(f"\nBest genome after {num_frequency_steps*frequency} generations:\n{winner}")

    # Visualize
    visualize.draw_net(config, winner, view=False)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

if __name__ == "__main__":
    run(Path("./us_market_config"))


Curriculum step 0: now using 1 files.

 ****** Running generation 0 ****** 

         4448133 function calls (4448123 primitive calls) in 6.942 seconds

   Ordered by: cumulative time
   List reduced from 105 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.034    0.034    6.942    6.942 /Users/anshul/Documents/programming/Python/StockMarket/Neat_RL/game_environment.py:1024(call_internal)
      380    5.179    0.014    5.186    0.014 /Users/anshul/Documents/programming/Python/StockMarket/Neat_RL/game_environment.py:1042(<listcomp>)
        1    0.002    0.002    1.400    1.400 /Users/anshul/Documents/programming/Python/StockMarket/Neat_RL/game_environment.py:1030(<listcomp>)
     2000    0.483    0.000    1.398    0.001 /Users/anshul/Documents/programming/Python/StockMarket/.venv/lib/python3.10/site-packages/neat/nn/feed_forward.py:28(create)
     2000    0.059    0.000    0.564    0.000 /Users/anshul/Documents/progra

KeyboardInterrupt: 